In [1]:
import numpy as np
import networkx as nx
import pylab as plt
from scipy.stats import norm,uniform,binom,poisson
import random
from functools import reduce
from itertools import product
from collections import deque

In [ ]:
def float_range(stop, start=0, step=1):
    while start < stop:
        yield float(start)
        start += step 

def tuple_eq(tup1,tup2):
    return len(set(tup1).union(set(tup2))) == len(tup1) == len(tup2)
    
def tuple_rev(tup):
    return (tup[1],tup[0])

def tuple_int(tup1,tup2):
    return len(set(tup1).union(set(tup2))) < len(tup1) + len(tup2)

In [3]:
class Node:
    def __init__(self):
        self.id = 0 #id(node) for edge
        self.x = None
        self.y = None
        self.weight = 0
    
    def __key(self):
        return (self.x, self.y, self.id, self.weight)
    def __hash__(self) -> int:
        return hash(self.__key())
    def __eq__(self, o: object) -> bool:
        if isinstance(o, Node):
            return self.__key() == o.__key()
        return False
    def __repr__(self) -> str:
        return f"Node(({self.x},{self.y}), id {self.id}, weight {self.weight})"
    def __str__(self) -> str:
        return f"Node ({self.x},{self.y})||id {self.id}||w {self.weight}"
    def __getitem__(self, val = None):
        if val == 'id':
            return self.id
        if val == 'x' or val == 0:
            return self.x
        if val == 'y' or val == 1:
            return self.y
        if val == 'w':
            return self.weight
        return self.__key()
    def __setitem__(self, key, value):
        if key == 'id':
            self.id = value
        if key == 'x' or key == 0:
            self.x = value
        if key == 'y' or key == 1:
            self.y = value
        if key == 'w':
            self.weight = value

In [ ]:
class Node2:
    def __init__(self, id = {'x':None , 
                             'y':None, 
                             'key':None, 
                             'weight':0}):
        self.id = id
        self.x = id['x']
        self.y = id['y']
        self.weight = id['weight']
        self.key = id['key']
        
class Node3:
    def __init__(self, id = None):
        self.id = id
        self.attr = {}